# **PHI-HRT Pipeline**

_____________

Author: Jonas Sinjan (MPS) <br>
Contributor: Daniele Calchetti (MPS)

Reductions Steps

1. read in science data (+scaling) open path option + open for several scans at once
2. read in flat field (+scaling)- accepts only one flat field fits file
3. read in dark field (+scaling)
4. apply dark field (to only science - assumes flat is dark fielded)
5. option to clean flat field with unsharp masking (Stokes QUV, UV or V)
6. normalise flat field
7. apply flat field
8. prefilter correction
9. apply field stop
10. demodulate with const demod matrix
11. normalise to quiet sun
12. I -> Q,U,V cross talk correction
13. rte inversion with cmilos
       

## Step 1: Download files

(This requires the hrt_pipeline_env environment - or the correct modules installed with pip (see `requirements.txt`)

Options:

1. Can download manually yourself from the PHI Image Database: https://www2.mps.mpg.de/services/proton/phi/imgdb/

    Suggested filters on the database for HRT science data: 
    - KEYWORD DETECTOR = 'HRT' <br >
    - Filename\* like \*L1_phi-hrt-ilam_date\*
    
   Once you find the file you want use the Command Line: 
    
    ```
    wget --user yourusername --password yourpassword file_web_address_from_database
    gunzip file.gz
    ```
       
2. Use the provided script in 'downloads' folder (**Benefit: can download multiple files at once**): `downloads/download_from_db.py`

    Instructions:
  1. From the database find the files you wish to download
  2. Copy the 'Download File List' that the database will generate
  3. Paste into the `downloads/file_names.txt` file
  4. Create a `downloads/.env` file with your MPS Windows login: <br> 
      ```text=
      USER_NAME =
      PHIDATAPASSWORD =
      ```  
  5. Set the **target download folder** in the `download_from_db.py` file
  6. Run the file (will require dotenv python module to be installed - included in `hrt_pipeline_env`):
      ```text=
      python download_from_db.py
      ```


## Step 2: Create Input JSON (config) file

In [10]:
import json

#example

data_path = ['data_one.fits', 'data_two.fits', 'etc.fits...'] 

#can have as many files as you like, contintue the list

"""
IMPORTANT

- files will be procssed with the same flat, dark
- must have the same continuum position, pmp temp etc - checks will be run, and pipeline will exit if they fail
- if data is a limb image, the limb in the FOV must be known - ie N, S, E, W etc. before processing
"""

dir_data_path = '/path/to/your/data/folder'
data = [dir_data_path + i for i in data_path]

flat_path = '/path/to/your/flat.fits'
dark_path = '/path/to/your/dark.fits'

input_dict = {

    #input data
    'data_f' : data,                            #hrt pipeline allows multiple files at once to be processed 
    'flat_f' : flat_path,
    'dark_f' : dark_path,
    
    #input/output type + scaling
    'L1_input' : False,                         #ignore - in development
    'L1_8_generate': False,                     #ignore - in development
    'scale_data' : True,                        #leave as True if L1 data
    'accum_scaling' : True,                     #leave as True if L1 data
    'bit_conversion' : True,                    #leave as True if L1 data
    
    #reduction
    'dark_c' : True,                            #apply dark field correction
    'flat_c' : True,                            #apply flat field correction
    'norm_f' : True,                            #normalise the flats before used (DEFAULT: True)
    'clean_f' : True,                           #clean the flat fields with unsharp masking
    'sigma' : 59,                               #unsharp masking gaussian width
    'clean_mode' : "V",                         #options 'QUV', 'UV', 'V' for the unsharp masking
    'flat_states' : 24,                         #options 24 (DEFAULT), 4 (one each pol state), 6 (one each wavelength), 
    'prefilter_f': None,                        #provide the path for the prefilter .fits file
    'fs_c' : True,                              #apply the field stop
    'iss_off': False,                           #If iss switched off, set this to True, will coalign images - STRONGLY recomended to also then set VtoQU to True
    'demod' : True,                             #demodulate to create the stokes maps
    'norm_stokes' : True,                       #normalise the stokes maps to I_continuum
    'ItoQUV' : True,                            #apply I-> Q,U,V cross talk correction
    'VtoQU' : False,                            #apply V-> Q,U cross talk correction
    'ghost_c' : False,                          #if True, excludes ghost region for ItoQUV ctalk calculation 
    'rte' : False,                              #options: RTE', 'CE' (classical estimates), 'CE+RTE'
    'p_milos' : False,                          #DO NOT USE
    'cmilos_fits': False,                       #use cmilos with .fits IO - 16% speed up - but not calibrated with default CMILOS
    
    #output dir/filenames
    'out_dir' : None,                           #directory where you want the output files to go (string)
    'out_stokes_file' : False,                  #if True, will save the final stokes array to fits file(s)
    'out_stokes_filename' : None,               #if specific required otherwise will default to standard convention
    'out_rte_filename' : None,                  #if specific required otherwise will default to standard convention
    'config' : True,                            #if True, saves a copy of the input file with runtime in the output folder
    'out_intermediate': False,                  #if True, save intermediate steps to fits files in the output folder (OUT_STOKES_FILE MUST BE SET TO TRUE)
    'vers': "01",                               #desired version number, only 2 characters 01 -> 99, if not specified, '01' default
}

In [ ]:
##################################################################

#      CHANGE NAME OF CONFIG FILE + CREATE

##################################################################

#json.dump(input_dict, open(f"./input_jsons/name_your_input_file.json", "w")) #uncomment to run

# Step 3: Run the Pipeline

Two options
1. Run within notebook <br>
2. Run in Terminal

In [9]:
#Example RUN the pipe
%load_ext autoreload
%autoreload 2

#make sure the 'sophi_hrt_pipe' package is installed in your environment:
# 'pip install .' or 'conda develop .'

from sophi_hrt_pipe.hrt_pipe import phihrt_pipe

#insert name of your input file

stokes = phihrt_pipe('./input_jsons/name_of_your_input_file.json')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
-------------------------------------------------------------- 
PHI HRT data reduction software   
-------------------------------------------------------------- 
 
-->>>>>>> Reading Data
Input contains 1 scan(s)
Dividing by number of accumulations: 20
Continuum position at wave:  0
This scan has been flipped in the Y axis to conform to orientation standards. 
 File: /data/slam/home/sinjan/fits_files/solo_L1_phi-hrt-ilam_20210914T071515_V202110260809C_0149140401.fits
All the scan(s) have the same dimension
All the scan(s) have the same continuum wavelength position
All the scan(s) have the same PMP Temperature Set Point: 50
Data shape is (2048, 2048, 24, 1)
All the scan(s) have the same IMGDIRX keyword: YES
Data reshaped to:  (2048, 2048, 4, 6, 1)
-------------------------------------------------------------- 
------------ Load science data time: 0.589 seconds 
--------------------------------------

In [ ]:
#Or can run in terminal (either through jupyter or yourself):
import os
cwd = os.getcwd()
os.chdir(cwd + './src/sophi_hrt_pipe/')

#####################################################################

#       CHANGE INPUT FILE LOC IN 'phihrt_pipe' function in 'run.py'

#####################################################################

!python run.py

os.chdir(cwd)